# Projection issues in SSEN data

I've noticed points that are skewed north west when comparing different SSEN data on a 
map. I don't know why yet, but I do know I've done some coordinate system conversions,
so I want to make sure it's not an error on my side.

This notebook is to explore it.

First load the substation data and the transformer load model and put them onto a map
together.

Data is from:
- https://data.ssen.co.uk/@ssen-distribution/ssen-substation-data
- https://data.ssen.co.uk/@ssen-distribution/transformer-load-model

Note: when viewing on GitHub this notebook won't display the maps, GH's viewer is just a
preview. You can use nbviewer.org to see it: https://nbviewer.org/github/centre-for-ai-and-climate/weave/blob/ssen-substation-locations/experiments/ssen-substation-projections.ipynb

In [79]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np

DOWNLOADS_PATH = "~/Downloads"
# Loading the map is slow when the data is large, this is a useful enough smaller area
example_rect = bbox = [-2.45, 50.7, -2.35, 50.9]

# The most comprehensive substation data file - what I assume is correct
ssen_substation_data_df = pd.read_csv(os.path.join(DOWNLOADS_PATH, "ssen-substation-data.csv"), usecols=["Location X m", "Location Y m", "Type", "Class", "Number", "Status"], dtype={"Type": str, "Class": str, "Number": str, "Status": str, "Location X m": np.float64, "Location Y m": np.float64})
ssen_substation_data_gdf = gpd.GeoDataFrame(ssen_substation_data_df, geometry=gpd.points_from_xy(ssen_substation_data_df["Location X m"], ssen_substation_data_df["Location Y m"]), crs="EPSG:27700")
# Convert to WGS84 explicitly for display, in case I need to investigate this
ssen_substation_data_gdf = ssen_substation_data_gdf.to_crs("EPSG:4326")
# We're not interested in primary substations, they're not in the other data
ssen_substation_data_gdf = ssen_substation_data_gdf[~(ssen_substation_data_gdf["Type"] == "Primary")]
ssen_substation_data_gdf = ssen_substation_data_gdf.clip(example_rect)
m = ssen_substation_data_gdf.explore(color="orange", marker_kwds={"radius": 10, "fill": False}, location=(50.7962, -2.4189), zoom_start=15)

# The "transformer load model" we'd like to use but is a bit suspect
transformer_load_model_df = pd.read_csv(os.path.join(DOWNLOADS_PATH, "onedrive_1_01-08-2024/SEPD_transformers_open_data_with_nrn.csv"), usecols=["full_nrn", "latitude", "longitude"])
transformer_load_model_df.drop_duplicates(subset=["full_nrn"], inplace=True)
transformer_load_model_df.set_index("full_nrn", inplace=True)
transformer_load_model_df.dropna(subset=["latitude", "longitude"], inplace=True)
transformer_load_model_gdf = gpd.GeoDataFrame(transformer_load_model_df, geometry=gpd.points_from_xy(transformer_load_model_df["longitude"], transformer_load_model_df["latitude"]), crs="EPSG:4326")
transformer_load_model_gdf = transformer_load_model_gdf.clip(example_rect)
transformer_load_model_gdf.explore(m=m, color="blue", marker_kwds={"radius": 5})

display(m)

This demonstrates the problem, there are lots of substations which seem to match in
each dataset but which are skewed a bit south east.

However, I had to convert one data set from 27700 to 4326, so it could be that process 
which has skewed them.

Lets test the raw data more specifically to isolate that problem.

Picking a random substation which appears to have a skewed counterpart:

In [80]:
transformer_load_model_df.loc["7604_003_120"]

latitude     50.895833
longitude    -2.595055
Name: 7604_003_120, dtype: float64

The substation data says it's counterpart is located at:

X 358157.019
Y 110935.03

Which [OS transforms](https://www.ordnancesurvey.co.uk/gps/transformation/) to:

50.8963613993
2.5963442266

Which looks about right, it's about 0.0008, 0.0007 different which is in the order of
10 meters.

Therefore, I think the data itself is already skewed - my conversions are not adding to
that. From some reading, this is quite common when converting from 27700 -> 4326 without
the official "grid" which applies some micro-transformation of coordinates for each KM
grid square in the UK.

Lets see if we can undo this, as a way to test the hypothesis further. If we convert
the data back into 27700 without the grid shift, then try again with the grid shift, it
should align the points better.

In [81]:
# We need to isolate those coordinate transforms - simply using geopands to_crs will
# use the best transformer available every time.
from pyproj.transformer import TransformerGroup
tg = TransformerGroup("EPSG:27700", "EPSG:4326")
# tg is a list of possible transformers. At first, I did not have the "best" one - it
# knew about it and where to get it, but it needs downloading. It's the OSNT15 file:
# https://www.ordnancesurvey.co.uk/documents/gps/updated-transformations-uk-ireland-geoid-model.pdf
# See https://pyproj4.github.io/pyproj/stable/transformation_grids.html for more info
# I explored the outputs of all the transformers to see which one was closest to the
# original data and settled on the last one named "ballpark" as the most likely.

# This is probably the most reliable way to make sure the OSNT15 file is downloaded
# rather than trying to install it some other way
if not tg.best_available:
    tg.download_grids()
    tg = TransformerGroup("EPSG:27700", "EPSG:4326")
    assert tg.best_available


best_transformer = tg.transformers[0]
best_transformer

<Concatenated Operation Transformer: pipeline>
Description: Inverse of British National Grid + OSGB36 to WGS 84 (9)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.01, 49.75, 2.01, 61.01)

In [82]:
worst_transformer = tg.transformers[-1]
worst_transformer

<Concatenated Operation Transformer: pipeline>
Description: Inverse of British National Grid + Ballpark geographic offset from OSGB36 to WGS 84
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)

In [83]:
best_transformer.transform(358157.019, 110935.03)

(50.89636138973284, -2.596344231679267)

In [84]:
worst_transformer.transform(358157.019, 110935.03)

(50.89582432668474, -2.595039962705485)

In [85]:
tg_inverse = TransformerGroup("EPSG:4326", "EPSG:27700")
(x, y) = tg_inverse.transformers[-1].transform(50.895833, -2.595055)
tg.transformers[0].transform(x, y)

(50.89637006194477, -2.596359267607151)

Lets map that for all the data and see how it stacks up

In [86]:
# Try to "fix" SSENs badly converted locations in the transformer load model
# My hypothesis is that they started with EPSG:27700 and converted to EPSG:4326, but
# without the right datum/geoid or whatever you call it.
# We'll try, turning them back into EPSG:27700 using something similar to their original
# conversion (but in reverse) to get something close to the original Eastings and
# Northings. Then try to convert those Easting and Northings back to EPSG:4326 using a
# more accurate transformation.

from pyproj.transformer import TransformerGroup
from shapely import Point
# These lists are sorted best -> worst for accuracy
# We need always_xy=True because otherwise the transformers respect the ordering of the
# CRSs which gets confusing when converting between EPSG:4326 and EPSG:27700
ssen_inverse = TransformerGroup("EPSG:4326", "EPSG:27700", always_xy=True).transformers[-1]
osgb36_to_wgs84 = TransformerGroup("EPSG:27700", "EPSG:4326", always_xy=True).transformers[0]

def fix_ssen_transforms(point):
    assert isinstance(point, Point)
    easting, northing = ssen_inverse.transform(point.x, point.y)
    x, y = osgb36_to_wgs84.transform(easting, northing)
    return Point(x, y)

transformer_load_model_gdf["fixed_geometry"] = transformer_load_model_gdf.geometry.apply(fix_ssen_transforms)
transformer_load_model_gdf.set_geometry("fixed_geometry", inplace=True)
transformer_load_model_gdf.explore(m=m, color="green", marker_kwds={"radius": 5})

This looks pretty good when we map it all out, so how do we make sure we're always using 
these specific transformers? Pulling the first and last out of a list doesn't seem very
dependable.

In [87]:
tg_inverse.transformers[-1].definition

'proj=pipeline step proj=axisswap order=2,1 step proj=unitconvert xy_in=deg xy_out=rad step proj=tmerc lat_0=49 lon_0=-2 k=0.9996012717 x_0=400000 y_0=-100000 ellps=airy'

In [88]:
tg.transformers[0].definition

'proj=pipeline step inv proj=tmerc lat_0=49 lon_0=-2 k=0.9996012717 x_0=400000 y_0=-100000 ellps=airy step proj=hgridshift grids=uk_os_OSTN15_NTv2_OSGBtoETRS.tif step proj=unitconvert xy_in=rad xy_out=deg step proj=axisswap order=2,1'

In [89]:
from pyproj import Transformer
bad_transformer = Transformer.from_pipeline('proj=pipeline step proj=axisswap order=2,1 step proj=unitconvert xy_in=deg xy_out=rad step proj=tmerc lat_0=49 lon_0=-2 k=0.9996012717 x_0=400000 y_0=-100000 ellps=airy')
good_transformer = Transformer.from_pipeline('proj=pipeline step inv proj=tmerc lat_0=49 lon_0=-2 k=0.9996012717 x_0=400000 y_0=-100000 ellps=airy step proj=hgridshift grids=uk_os_OSTN15_NTv2_OSGBtoETRS.tif step proj=unitconvert xy_in=rad xy_out=deg step proj=axisswap order=2,1')

In [90]:
(easting, northing) = bad_transformer.transform(50.895833, -2.595055)
good_transformer.transform(easting, northing)

(50.89637006194477, -2.596359267607151)

That matches, so we can build a specific transformer using a proj pipeline string